# GP2 Tutorial - Module 6 - Example 2 - Single gene analyses

`GP2 ❤️ Open Science 😍`

- **Module:** GP2 Demo
- **Authors:** Sara Bandres-Ciga, Hampton Leonard, and Mary Makarious on behalf of the Global Parkinson's Genetics Program
- **Estimated Computation and Runtime:**
    - **Estimated Specifications:** Default; 1 CPU, 3.75 GB Memory, 50 GB Persistent Disk Size
    - **Estimated Runtime:** 30 min.
- **Date Last Updated:** 17-JULY-2022
    - **Update Description:** Finished notebook!

---
### Quick Description
These notebooks serve as a beginner's introduction to Terra, GP2, and AMP-PD data to help you get comfortable navigating the spaces, manage your costs, upload your own data, and begin running some simple analyses.


### Course Summary
- **Module 1:** Intro + demo Terra
- **Module 2:** Introduction to current AMP and GP2 data + managing costs
- **Module 3:** How to upload, access, and copy over data 
- **Module 4:** How to interact with clinical data + make mini covariate files
- **Module 5:** Analysis example 1: Run PRS with PD known hits in Non-Euro pops and data viz + save results
- **Module 6:** Analysis example 2: How to extract a gene, annotate it, run burden, and get hmz and compound hets + save results **(this notebook!)**
- **Module 7:** WDL workflows: What they are + When to use + Quick example overview 

--- 
## Notebook Summary 
 - Set up environment
 - Install packages
 - Copy data from GP2 Tier 2 directory to workspace
 - Extract LRRK2 and GBA genes and variants from GP2 release 1 data
 - Annotate variants
 - Run burden analyses
 - Explore PARK2 homozygotes and compound heterozygotes
 
## Workflow

### [Getting Started](#1)
This section goes through: 
- Set up environment
- Install packages
- Copy data from GP2 Tier 2 bucket to workspace

### [Gene extraction](#2)
This section goes through:
- Extract LRRK2 and GBA genes from GP2 release 2 data (AJ population) using plink 1.9 and plink 2.0
- Extract LRRK2 G2019S and GBA N370S genes from GP2 release 1 data (AJ population) using plink 1.9 and plink 2.0

### [Annotation](#3)
This section goes through:
- Annotate LRRK2 and GBA genes from GP2 release 2 data (AJ population) using ANNOVAR
- Format files
- Extract coding variants

### [Burden analyses](#4)
This section goes through:
- Burden analyses including LRRK2 coding variants and interpretation

### [Homozygotes/Compound Heterozygotes](#5)
This section goes through:
- Exploring homozygotes and compound het PARK2 carriers

### [Save out results](#6)
This section goes through:
- Tranfer output files into the bucket


## Set up environment

In [2]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## AMP-PD v2.5
## Explicitly define release v2.5 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2021_v2-5release_0510'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_GATK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'gatk')

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')

print('AMP-PD v2.5')
print(f'Path to AMP-PD v2.5 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v2.5 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print('')

## GP2 v2.0
GP2_RELEASE_PATH = 'gs://gp2tier2/release2_06052022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'

GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v2.0')
print(f'Path to GP2 v2.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v2.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v2.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')



Billing and Workspace
Workspace Name: GP2 Bioinformatics Course 2
Billing Project: terra-9b559320
Workspace Bucket, where you can upload and download data: gs://fc-c04486b2-8d7e-4359-a607-63643e9a7914

AMP-PD v2.5
Path to AMP-PD v2.5 Clinical Data: gs://amp-pd-data/releases/2021_v2-5release_0510/clinical
Path to AMP-PD v2.5 WGS Data: gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs/plink

GP2 v2.0
Path to GP2 v2.0 Clinical Data: gs://gp2tier2/release2_06052022/clinical_data
Path to GP2 v2.0 Raw Genotype Data: gs://gp2tier2/release2_06052022/raw_genotypes
Path to GP2 v2.0 Imputed Genotype Data: gs://gp2tier2/release2_06052022/imputed_genotypes


In [4]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

## Install packages

In [5]:
%%capture
%%bash

# Install plink 1.9
cd /home/jupyter/
if test -e /home/jupyter/plink; then

echo "Plink is already installed in /home/jupyter/"
else
echo "Plink is not installed"
cd /home/jupyter

wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

unzip -o plink_linux_x86_64_20190304.zip
mv plink plink1.9

fi

In [6]:
%%capture
%%bash

# Install plink 2.0
cd /home/jupyter/
if test -e /home/jupyter/plink2; then

echo "Plink2 is already installed in /home/jupyter/"
else
echo "Plink2 is not installed"
cd /home/jupyter/

wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

unzip -o plink2_linux_x86_64_latest.zip

fi

In [7]:
%%bash

# chmod plink 1.9 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink1.9

In [8]:
%%bash

# chmod plink 2.0 to make sure you have permission to run the program
chmod u+x /home/jupyter/plink2

In [9]:
%%capture
%%bash

# Install ANNOVAR: We are adding the download link after registration on the annovar website
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/annovar; then

echo "annovar is already installed in /home/jupyter/notebooks"
else
echo "annovar is not installed"
cd /home/jupyter/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

In [10]:
%%capture
%%bash

# Install ANNOVAR: Download resources for annotation

cd /home/jupyter/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/


In [11]:
%%bash
rm -r /home/jupyter/rvtests
#Install RVTESTS: Option 1 (~15min)
if test -e ~/home/jupyter/rvtests; then

echo "rvtests is already installed"
else
echo "rvtests is not installed"
cd /home/jupyter/

git clone https://github.com/zhanxw/rvtests

fi

rvtests is not installed


Cloning into 'rvtests'...


In [13]:
#Install RVTESTS: Option 2 (from the executable previously uploaded to the workspace bucket)
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/ /home/jupyter')

Executing: gsutil -u terra-9b559320 -m cp -r gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/ /home/jupyter


Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/bgen2vcf...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/combineKinship...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/bgen2vcf.d...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/bgenFileInfo.d...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/bgenFileInfo...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/combineKinship.d...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/createBCF2Index...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/createBCF2Index.d...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/createVCFIndex...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/createVCFIndex.d...
Copying gs://fc-secure-e7ee53f9-e643-4d4b-b4b8-955c8ad7b23e/executable/explainCSI1...
Copying gs://fc-secure-e7ee5

In [17]:
%%bash 

# Change permissions
chmod 777 /home/jupyter/executable/rvtest

In [18]:
%%bash

/home/jupyter/executable/rvtest --help

Basic Input/Output
                 --inVcf: Input VCF File
                --inBgen: Input BGEN File
          --inBgenSample: Input Sample IDs for the BGEN File
                 --inKgg: Input KGG File
                   --out: Output prefix
             --outputRaw: Output genotypes, phenotype, covariates(if any); and
                          collapsed genotype to tabular files
Specify Covariate
                 --covar: Specify covariate file
            --covar-name: Specify the column name in covariate file to be
                          included in analysis
                   --sex: Include sex (5th column in the PED file) as a covariate
Specify Phenotype
                 --pheno: Specify phenotype file
         --inverseNormal: Transform phenotype like normal distribution
--useResidualAsPhenotype: Fit covariate ~ phenotype, use residual to replace
                          phenotype
                --mpheno: Specify which phenotype column to read (default: 1);
            --p

In [19]:
# Copy Reflat_38 file into workspace
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {WORKSPACE_BUCKET}/refFlat.txt /home/jupyter')

Executing: gsutil -u terra-9b559320 -m cp gs://fc-c04486b2-8d7e-4359-a607-63643e9a7914/refFlat.txt /home/jupyter


Copying gs://fc-c04486b2-8d7e-4359-a607-63643e9a7914/refFlat.txt...
/ [1/1 files][ 21.2 MiB/ 21.2 MiB] 100% Done                                    
Operation completed over 1 objects/21.2 MiB.                                     


## Copy GP2 data from GP2 Tier 2 directory to workspace

In [20]:
# Create a folder on your workspace
print("Making a working directory")
WORK_DIR = f'/home/jupyter/LRRK2_GBA/'
shell_do(f'mkdir -p {WORK_DIR}') # f' stands for f-string which contains expressions inside brackets

Making a working directory


Executing: mkdir -p /home/jupyter/LRRK2_GBA/


In [21]:
# Check directory where GP2 Tier 2 data is
print("List available imputed genotype information in GP2 (broken down by ancestry)")
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

List available imputed genotype information in GP2 (broken down by ancestry)


Executing: gsutil -u terra-9b559320 ls gs://gp2tier2/release2_06052022/imputed_genotypes


gs://gp2tier2/release2_06052022/imputed_genotypes/AAC/
gs://gp2tier2/release2_06052022/imputed_genotypes/AFR/
gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/
gs://gp2tier2/release2_06052022/imputed_genotypes/AMR/
gs://gp2tier2/release2_06052022/imputed_genotypes/CAS/
gs://gp2tier2/release2_06052022/imputed_genotypes/EAS/
gs://gp2tier2/release2_06052022/imputed_genotypes/EUR/
gs://gp2tier2/release2_06052022/imputed_genotypes/SAS/


In [22]:
# Copy files from bucket to yout workspace - Here we will use EAS as an example
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/AJ {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/projected_pcs.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_CLINICAL_RELEASE_PATH}/master_key_release2_final.csv {WORK_DIR}')

Executing: gsutil -u terra-9b559320 -m cp -r gs://gp2tier2/release2_06052022/imputed_genotypes/AJ /home/jupyter/LRRK2_GBA/


Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr10_AJ_release2.pvar...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr10_AJ_release2.psam...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr10_AJ_release2.pgen...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr10_AJ_release2.log...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr11_AJ_release2.log...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr11_AJ_release2.pgen...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr11_AJ_release2.psam...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr11_AJ_release2.pvar...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr12_AJ_release2.log...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr12_AJ_release2.pgen...
Copying gs://gp2tier2/release2_06052022/imputed_genotypes/AJ/chr12_AJ_release2.psam...
Copying gs://gp2tier2/release2_06052022/impute

Executing: gsutil -u terra-9b559320 -m cp gs://gp2tier2/release2_06052022/meta_data/projected_pcs.csv /home/jupyter/LRRK2_GBA/


Copying gs://gp2tier2/release2_06052022/meta_data/projected_pcs.csv...
/ [1/1 files][ 11.0 MiB/ 11.0 MiB] 100% Done                                    
Operation completed over 1 objects/11.0 MiB.                                     


Executing: gsutil -u terra-9b559320 -m cp gs://gp2tier2/release2_06052022/clinical_data/master_key_release2_final.csv /home/jupyter/LRRK2_GBA/


Copying gs://gp2tier2/release2_06052022/clinical_data/master_key_release2_final.csv...
/ [1/1 files][656.7 KiB/656.7 KiB] 100% Done                                    
Operation completed over 1 objects/656.7 KiB.                                    


# Data formatting on covariate file

In [23]:
# GP2 Tier 2 covariate file (master_key_release*)
WORK_DIR = '/home/jupyter/LRRK2_GBA/'
FULL_PATH = WORK_DIR + 'master_key_release2_final.csv'
cov = pd.read_csv(FULL_PATH)
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'Phenotype', 'sex_for_qc', 'age',
       'age_of_onset', 'age_at_diagnosis', 'age_at_death', 'race_for_qc',
       'family_history_for_qc', 'region_for_qc'],
      dtype='object')

In [25]:
cov_reduced = cov[['GP2sampleID', 'sex_for_qc', 'age', 'Phenotype']]
cov_reduced.rename(columns = {'GP2sampleID':'IID'}, inplace = True)
# cov_reduced.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
# GP2 Tier 2 covariate file (projected PCs)
WORK_DIR = '/home/jupyter/LRRK2_GBA/'
FULL_PATH = WORK_DIR + 'projected_pcs.csv'
cov2 = pd.read_csv(FULL_PATH)
cov2.columns

Index(['FID', 'IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8',
       'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17',
       'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26',
       'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35',
       'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44',
       'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50', 'label'],
      dtype='object')

In [27]:
# Merge datasets by IID
WORK_DIR = '/home/jupyter/LRRK2_GBA/'
FULL_PATH = WORK_DIR + 'projected_pcs.csv'
cov2 = pd.read_csv(FULL_PATH)
cov_final = cov2.merge(cov_reduced, on='IID') # Horizontal.

cov_for_rv_test_df = cov_final
cov_for_rv_test_df.insert(0,'fid',cov_for_rv_test_df['FID'])
cov_for_rv_test_df.insert(1,'iid',cov_for_rv_test_df['IID'])
cov_for_rv_test_df.insert(2,'fatid',0)
cov_for_rv_test_df.insert(3,'matid',0)
cov_for_rv_test_df.insert(4,'sex',cov_for_rv_test_df['sex_for_qc'])

cov_for_rv_test_df.loc[cov_for_rv_test_df["sex"] == "Male", "sex"] = 1
cov_for_rv_test_df.loc[cov_for_rv_test_df["sex"] == "Female", "sex"] = 2
cov_for_rv_test_df.loc[cov_for_rv_test_df["Phenotype"] == "Control", "Phenotype"] = 1
cov_for_rv_test_df.loc[cov_for_rv_test_df["Phenotype"] == "PD", "Phenotype"] = 2
cov_for_rv_test_df.loc[cov_for_rv_test_df["Phenotype"] == "Other", "Phenotype"] = 0
cov_for_rv_test_df.loc[cov_for_rv_test_df["Phenotype"] == "Not Reported", "Phenotype"] = 0

cov_for_rv_test_df.head()
cov_for_rv_test_df.to_csv('/home/jupyter/LRRK2_GBA/gp2_covs_merged.csv', header=True, index=False)
cov_for_rv_test_df.to_csv('/home/jupyter/LRRK2_GBA/gp2_covs_merged.txt', header=True, index=False, sep=" ")

pheno_for_rv_test_df = cov_for_rv_test_df[['fid','iid','Phenotype']]
pheno_for_rv_test_df.to_csv('/home/jupyter/LRRK2_GBA/gp2_pheno_merged.csv', header=True, index=False)
pheno_for_rv_test_df.to_csv('/home/jupyter/LRRK2_GBA/gp2_pheno_merged.txt', header=True, index=False, sep=" ")

cov_for_rv_test_df['Phenotype'].value_counts()

2    5276
1    3396
0       8
Name: Phenotype, dtype: int64

## Extract LRRK2 and GBA genes from GP2 release 2 data

In [28]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# LRRK2: Specify gene boundaries in hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/plink2 \
--pfile chr12_AJ_release2 \
--chr 12 \
--from-bp 40196744  \
--to-bp 40369285 \
--make-bed \
--out LRRK2_AJ

PLINK v2.00a3.3LM 64-bit Intel (3 Jun 2022)    www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to LRRK2_AJ.log.
Options in effect:
  --chr 12
  --from-bp 40196744
  --make-bed
  --out LRRK2_AJ
  --pfile chr12_AJ_release2
  --to-bp 40369285

Start time: Mon Aug  1 01:29:10 2022
3679 MiB RAM detected; reserving 1839 MiB for main workspace.
Using 1 compute thread.
750 samples (270 females, 480 males; 750 founders) loaded from
chr12_AJ_release2.psam.
526642 variants loaded from chr12_AJ_release2.pvar.
1 binary phenotype loaded (550 cases, 200 controls).
798 variants remaining after main filters.
Writing LRRK2_AJ.fam ... done.
Writing LRRK2_AJ.bim ... done.
Writing LRRK2_AJ.bed ... 0%done.
End time: Mon Aug  1 01:29:11 2022


In [29]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# Convert binary files into vcf file
/home/jupyter/plink1.9 \
--bfile LRRK2_AJ \
--recode vcf-fid \
--out LRRK2_AJ

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to LRRK2_AJ.log.
Options in effect:
  --bfile LRRK2_AJ
  --out LRRK2_AJ
  --recode vcf-fid

3679 MB RAM detected; reserving 1839 MB for main workspace.
798 variants loaded from .bim file.
750 people (480 males, 270 females) loaded from .fam.
750 phenotype values loaded from .fam.
Using 1 thread.
Before main variant filters, 750 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%66%67%68%69%70%71%72%73%74%75%

In [30]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# How about LRRK2 G2019S?
/home/jupyter/plink2 \
--pfile chr12_AJ_release2 \
--chr 12 \
--from-bp 40340400  \
--to-bp 40340400 \
--make-bed \
--out LRRK2_AJ_G2019S

PLINK v2.00a3.3LM 64-bit Intel (3 Jun 2022)    www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to LRRK2_AJ_G2019S.log.
Options in effect:
  --chr 12
  --from-bp 40340400
  --make-bed
  --out LRRK2_AJ_G2019S
  --pfile chr12_AJ_release2
  --to-bp 40340400

Start time: Mon Aug  1 01:31:10 2022
3679 MiB RAM detected; reserving 1839 MiB for main workspace.
Using 1 compute thread.
750 samples (270 females, 480 males; 750 founders) loaded from
chr12_AJ_release2.psam.
526642 variants loaded from chr12_AJ_release2.pvar.
1 binary phenotype loaded (550 cases, 200 controls).
1 variant remaining after main filters.
Writing LRRK2_AJ_G2019S.fam ... done.
Writing LRRK2_AJ_G2019S.bim ... done.
Writing LRRK2_AJ_G2019S.bed ... 0%done.
End time: Mon Aug  1 01:31:10 2022


In [31]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# GBA: Specify gene boundaries in hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/plink2 \
--pfile chr1_AJ_release2 \
--chr 1 \
--from-bp 155234452  \
--to-bp 155244699 \
--make-bed \
--out GBA_AJ

PLINK v2.00a3.3LM 64-bit Intel (3 Jun 2022)    www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GBA_AJ.log.
Options in effect:
  --chr 1
  --from-bp 155234452
  --make-bed
  --out GBA_AJ
  --pfile chr1_AJ_release2
  --to-bp 155244699

Start time: Mon Aug  1 01:31:57 2022
3679 MiB RAM detected; reserving 1839 MiB for main workspace.
Using 1 compute thread.
750 samples (270 females, 480 males; 750 founders) loaded from
chr1_AJ_release2.psam.
859027 variants loaded from chr1_AJ_release2.pvar.
1 binary phenotype loaded (550 cases, 200 controls).
23 variants remaining after main filters.
Writing GBA_AJ.fam ... done.
Writing GBA_AJ.bim ... done.
Writing GBA_AJ.bed ... 0%done.
End time: Mon Aug  1 01:31:58 2022


In [32]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# Convert binary files into vcf file
/home/jupyter/plink1.9 \
--bfile GBA_AJ \
--recode vcf-fid \
--out GBA_AJ

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GBA_AJ.log.
Options in effect:
  --bfile GBA_AJ
  --out GBA_AJ
  --recode vcf-fid

3679 MB RAM detected; reserving 1839 MB for main workspace.
23 variants loaded from .bim file.
750 people (480 males, 270 females) loaded from .fam.
750 phenotype values loaded from .fam.
Using 1 thread.
Before main variant filters, 750 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%65%66%67%68%69%70%71%72%73%74%75%76%

In [33]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# How about GBA N370S?
/home/jupyter/plink2 \
--pfile chr1_AJ_release2 \
--chr 1 \
--from-bp 155235843  \
--to-bp 155235843 \
--make-bed \
--out GBA_AJ_N370S

PLINK v2.00a3.3LM 64-bit Intel (3 Jun 2022)    www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to GBA_AJ_N370S.log.
Options in effect:
  --chr 1
  --from-bp 155235843
  --make-bed
  --out GBA_AJ_N370S
  --pfile chr1_AJ_release2
  --to-bp 155235843

Start time: Mon Aug  1 01:33:11 2022
3679 MiB RAM detected; reserving 1839 MiB for main workspace.
Using 1 compute thread.
750 samples (270 females, 480 males; 750 founders) loaded from
chr1_AJ_release2.psam.
859027 variants loaded from chr1_AJ_release2.pvar.
1 binary phenotype loaded (550 cases, 200 controls).
1 variant remaining after main filters.
Writing GBA_AJ_N370S.fam ... done.
Writing GBA_AJ_N370S.bim ... done.
Writing GBA_AJ_N370S.bed ... 0%done.
End time: Mon Aug  1 01:33:11 2022


In [34]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/rvtests/third/tabix-0.2.6/

bgzip LRRK2_AJ.vcf
bgzip GBA_AJ.vcf

tabix -f -p vcf LRRK2_AJ.vcf.gz
tabix -f -p vcf GBA_AJ.vcf.gz

[bgzip] can't create LRRK2_AJ.vcf.gz: File exists
[bgzip] can't create GBA_AJ.vcf.gz: File exists


In [37]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
ls $WORK_DIR/*vcf*

/home/jupyter/LRRK2_GBA/AJ/GBA_AJ.annovar.hg38_multianno.vcf
/home/jupyter/LRRK2_GBA/AJ/GBA_AJ.vcf
/home/jupyter/LRRK2_GBA/AJ/LRRK2_AJ.annovar.hg38_multianno.vcf
/home/jupyter/LRRK2_GBA/AJ/LRRK2_AJ.coding.vcf
/home/jupyter/LRRK2_GBA/AJ/LRRK2_AJ.vcf


## Annotate variants

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

perl /home/jupyter/annovar/table_annovar.pl LRRK2_AJ.vcf.gz /home/jupyter/annovar/humandb/ -buildver hg38 \
-out LRRK2_AJ.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# Make a header
head -1 LRRK2_AJ.annovar.hg38_multianno.txt > header_LRRK2.txt

# Count number of columns in the header
colct="$(wc -w header_LRRK2.txt | cut -f1 -d' ')"

# Extract columns in the header (we want to get rid of all the indv columns without a header)
cut -f1-$colct LRRK2_AJ.annovar.hg38_multianno.txt > LRRK2_AJ.trimmed.annotation.txt

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# Extract exonic variants
awk '$6=="exonic" {print}' LRRK2_AJ.trimmed.annotation.txt > LRRK2_AJ.trimmed.coding.annotation.txt

# Extract columns chr, pos, pos, gene
awk '{print $1" "$2" "$2" "$7}' LRRK2_AJ.trimmed.coding.annotation.txt > LRRK2_AJ.trimmed.coding.SNPs.txt
head LRRK2_AJ.trimmed.coding.SNPs.txt
wc -l LRRK2_AJ.trimmed.coding.SNPs.txt

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

perl /home/jupyter/annovar/table_annovar.pl GBA_AJ.vcf.gz /home/jupyter/annovar/humandb/ -buildver hg38 \
-out GBA_AJ.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

head -1 GBA_AJ.annovar.hg38_multianno.txt > header_GBA.txt
colct="$(wc -w header_GBA.txt | cut -f1 -d' ')"
cut -f1-$colct GBA_AJ.annovar.hg38_multianno.txt > GBA_AJ.trimmed.annotation.txt

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

# Extract exonic variants
awk '$6=="exonic" {print}' GBA_AJ.trimmed.annotation.txt > GBA_AJ.trimmed.coding.annotation.txt

# Extract columns chr, pos, pos, gene
awk '{print $1" "$2" "$2" "$7}' GBA_AJ.trimmed.coding.annotation.txt > GBA_AJ.trimmed.coding.SNPs.txt
head GBA_AJ.trimmed.coding.SNPs.txt
wc -l GBA_AJ.trimmed.coding.SNPs.txt

## Run burden analyses

In [ ]:
%%bash

# Run burden analyses only using coding variants
WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

/home/jupyter/plink1.9 \
--bfile LRRK2_AJ \
--extract range LRRK2_AJ.trimmed.coding.SNPs.txt \
--recode vcf-iid \
--out LRRK2_AJ.coding

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

export PATH=$PATH:/home/jupyter/rvtests/third/tabix-0.2.6/
bgzip LRRK2_AJ.coding.vcf
tabix -f -p vcf LRRK2_AJ.coding.vcf.gz

In [ ]:
%%bash

WORK_DIR='/home/jupyter/LRRK2_GBA/AJ'
cd $WORK_DIR

/home/jupyter/executable/rvtest --noweb --hide-covar \
--out LRRK2_AJ_burden \
--burden cmc,zeggini,mb,fp,cmcWald --kernel skat,skato \
--inVcf LRRK2_AJ.coding.vcf.gz \
--pheno /home/jupyter/LRRK2_GBA/gp2_covs_merged.txt  \
--pheno-name Phenotype \
--geneFile /home/jupyter/refFlat.txt \
--covar /home/jupyter/LRRK2_GBA/gp2_covs_merged.txt \
--covar-name sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 \

# --out : Name of output 
# --burden cmc --kernel skato: tests to run 
# --inVcf : VCF file 
# --geneFile refFlat.txt
# --pheno :  covar file
# --pheno-name : column name with phenotype in file
# --covar : covar file
# --freqUpper : optional, MAF cut-off
# --covar-name : covariates, listed by column name, separated by commas (no spaces between commas)


In [ ]:
%%bash

WORK_DIR=/home/jupyter/LRRK2_GBA/AJ
ls $WORK_DIR/*assoc

## Explore PARK2 homozygotes and compound heterozygotes (coding variants only)

In [ ]:
# Create a folder on your workspace
print("Making a working directory")
WORK_DIR_2 = f'/home/jupyter/PARK2/AJ'
shell_do(f'mkdir -p {WORK_DIR_2}') # f' stands for f-string which contains expressions inside brackets

In [ ]:
%%bash -s $WORK_DIR

WORK_DIR=/home/jupyter/LRRK2_GBA/AJ
WORK_DIR_2=/home/jupyter/PARK2/AJ

cd $1

/home/jupyter/plink2 \
--pfile $WORK_DIR/chr6_AJ_release2 \
--chr 6 \
--from-bp 161347417   \
--to-bp 162727775 \
--make-bed \
--out $WORK_DIR_2/PARK2_AJ

In [ ]:
%%bash

WORK_DIR=/home/jupyter/LRRK2_GBA/AJ
WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

/home/jupyter/plink1.9 \
--bfile PARK2_AJ \
--recode vcf-iid \
--out PARK2_AJ

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

export PATH=$PATH:/home/jupyter/rvtests/third/tabix-0.2.6/
bgzip PARK2_AJ.vcf
tabix -f -p vcf PARK2_AJ.vcf.gz

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

perl /home/jupyter/annovar/table_annovar.pl PARK2_AJ.vcf.gz /home/jupyter/annovar/humandb/ -buildver hg38 \
-out PARK2_AJ.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
-nastring . \
-vcfinput

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

head -1 PARK2_AJ.annovar.hg38_multianno.txt > header_PARK2_AJ.txt
colct="$(wc -w header_PARK2_AJ.txt | cut -f1 -d' ')"
cut -f1-$colct PARK2_AJ.annovar.hg38_multianno.txt > PARK2_AJ.trimmed.annotation.txt

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

# Extract exonic variants
awk '$6=="exonic" {print}' PARK2_AJ.trimmed.annotation.txt > PARK2_AJ.trimmed.coding.annotation.txt
awk '{print $1" "$2" "$2" "$7}' PARK2_AJ.trimmed.coding.annotation.txt > PARK2_AJ.trimmed.coding.SNPs.txt

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

# Head file and count lines 
head PARK2_AJ.trimmed.coding.SNPs.txt
wc -l PARK2_AJ.trimmed.coding.SNPs.txt

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ/
cd $WORK_DIR_2

# Extract only coding variants
/home/jupyter/plink1.9 \
--bfile PARK2_AJ \
--extract range PARK2_AJ.trimmed.coding.SNPs.txt \
--recode A \
--out PARK2_AJ.coding

In [ ]:
WORK_DIR_2 = f'/home/jupyter/PARK2/AJ/'
FULL_PATH = WORK_DIR_2 + 'PARK2_AJ.coding.raw'
PARK2 = pd.read_csv(FULL_PATH, sep = " ")
PARK2.columns
# PARK2.head()

In [ ]:
PARK2.shape

In [ ]:
# Identifying individuals homozygotes for the alternate allele
homo = PARK2[(PARK2['chr6:161360193:C:T_T'] == 2) | (PARK2['chr6:161386823:C:G_G'] == 2) | (PARK2['chr6:162201165:C:T_T'] == 2)]
homo = homo.dropna()
print(homo.shape)
# homo.head()          


In [ ]:
homo_info_control = homo[homo['PHENOTYPE'] == 1]
print(homo_info_control.shape)
homo_info_case = homo[homo['PHENOTYPE'] == 2]
print(homo_info_case.shape)

In [ ]:
# homo_info_control.head()

In [ ]:
homo_info_control.to_csv('PARK2.coding.homo.controls.txt', sep = '\t', index=False)

In [ ]:
# homo_info_case.head()

In [ ]:
homo_info_case.to_csv('PARK2.coding.homo.case.txt', sep = '\t', index=False)

In [ ]:
# Identifying individuals heterozygotes for the alternate allele
het = PARK2[(PARK2['chr6:161360193:C:T_T'] == 1) | (PARK2['chr6:161386823:C:G_G'] == 1) | (PARK2['chr6:162201165:C:T_T'] == 1)]
het = het.dropna()
print(het.shape)
# het.head() 

In [ ]:
het['compound'] = het['chr6:161360193:C:T_T'] + het['chr6:161386823:C:G_G'] + het['chr6:162201165:C:T_T']
het['compound'].value_counts()

In [ ]:
comp_het_info_case = het[(het['PHENOTYPE'] ==2) & (het['compound'] ==2)]
print(comp_het_info_case.shape)
comp_het_info_control = het[(het['PHENOTYPE'] ==1) & (het['compound'] ==2)]
print(comp_het_info_control.shape)

In [ ]:
comp_het_info_case.to_csv('/home/jupyter/PARK2/AJ/PARK2.coding.het.comp.case.txt', sep = '\t', index=True)

In [ ]:
comp_het_info_control.to_csv('/home/jupyter/PARK2/AJ/PARK2.coding.het.comp.controls.txt', sep = '\t', index=True)

# Save out results

In [ ]:
%%bash

WORK_DIR_2=/home/jupyter/PARK2/AJ
cd $WORK_DIR_2

gsutil cp PARK2.coding.het.comp.case.txt gs://fc-762ef56f-d73b-4327-8e97-3213cce671fb
gsutil cp PARK2.coding.het.comp.controls.txt gs://fc-762ef56f-d73b-4327-8e97-3213cce671fb

WORK_DIR=/home/jupyter/LRRK2_GBA/AJ
cd $WORK_DIR

gsutil cp *assoc gs://fc-762ef56f-d73b-4327-8e97-3213cce671fb